# 讓我們來計算熱點吧


    流程大概是這樣:
    1.照樣先把文章讀進來  分成tuple存起來
    2.然後每讀到一個點 就去mariaDB加一次那個點的count

    因為懶  每次就直接import可能會用到的東西

In [1]:
#encoding=utf-8
import jieba
import pandas as pd
import csv
import geocoder
import pymysql
import math
import numpy as np

    下面比較詳細的解說我放在runningTemplateFianl裡面  這邊就直接跑了
    

In [2]:
jieba.load_userdict("okinawa.txt")
df = pd.read_csv('okinawacontent.csv')
tt=[]
with open('okinawa.txt',encoding="UTF-8") as f:
    for i in f:
      tt.append(i.strip())
A=set()
B=[]
for partcontent in df['content']:
    words=jieba.cut(partcontent)
    for word in words:
        for ltt in tt:
            if word == ltt:
                A.add(ltt)
    if len(A)>1:
        B.append(list(A))
    A=set()
    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.983 seconds.
Prefix dict has been built succesfully.


In [3]:
for inner in B:
    print(inner)

['首里城', '読谷村', '國際通', '沖繩', '中頭郡', '名護市', '御菓子御殿']
['萬座毛', '國際通', '古宇利島', '沖繩', '波上宮', '波之上海灘', '那霸']
['沖繩', '那霸機場']
['沖繩', '國際通']
['沖繩', '首里城', '金城町石疊道']
['波上宮', '沖繩', '國際通']
['沖繩', '首里城']
['沖繩', '浦添市', '港川外人住宅']
['沖繩', '港川外人住宅']
['榮町市場', '沖繩']
['榮町市場', '沖繩']
['沖繩', '首里城', '國際通']
['沖繩', '國頭郡']
['沖繩', '首里城']
['古宇利大橋', '古宇利島', '沖繩']
['沖繩', '座喜味城']
['沖繩', '日落海灘', '美國村']
['沖繩', '那霸機場']
['沖繩', '那霸市', '那霸', '國際通']
['那霸', '沖繩']
['波上宮', '沖繩', '那霸', '國際通']
['金城町石疊道', '國場川', '萬座毛', '首里城', '新都心', '沖繩', '殘波岬燈塔', '那霸']
['勝連城跡', '古宇利大橋', '海中道路', '國際通', '沖繩', '普天間宮', '美國村', '那霸']
['古宇利大橋', '萬座毛', '國際通', '古宇利島', '沖繩', '名護市', '那霸', '古宇利']
['波上宮', '沖繩', '浦添大公園', '國際通']
['豐見城市', '沖繩', '瀨長島']
['日落海灘', '國際通', '波上宮', '奧武山公園', '沖繩', '浦添大公園']
['古宇利大橋', '沖繩', '美國村', '西松屋', '名護市', '浦添大公園']
['沖繩', '首里城', '那霸市', '國際通']
['沖繩', '那霸', '新都心', '那霸機場', '國際通']
['萬座毛', '永旺夢樂城沖繩來客夢', '沖繩', '美國村', '中頭郡', '那霸']
['那霸機場', '國際通', '沖繩', '美國村', '名護市', '那霸']
['沖繩', '美國村']
['沖繩', '美麗海水族館']
['海洋博公園', '奧武山公園', '沖繩', '美國村', '北谷美國村']

    準備來計算次數了
    

In [4]:
db = pymysql.connect("1.162.63.208","bdse0303","bdse0303","bdse03-travel",charset='utf8' )
cur=db.cursor() 

In [5]:
for tmp in B:
    for inner in tmp:
        sql='SELECT * FROM ginwaydu2 WHERE place="'+inner+'"'
        if cur.execute(sql)==1:
            sql2 = "UPDATE ginwaydu2  SET count = count + 1  WHERE place='"+inner+"'"
            cur.execute(sql2)
db.commit()
db.close()